**Before you start, make sure you use the T4 GPU processor (Runtime->Change Runtime-> T4GPU)**

In [ ]:
#Import drive
from google.colab import drive
#Mount Google Drive
ROOT="/content/drive"
drive.mount(ROOT, force_remount=True)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

cudnn.benchmark = True
plt.ion()   # interactive mode

Send the code to the GPU

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

**Guide for the HW**

This guide has part of the code needed for th homework. An understanding of the same will be of great help for the following exercises.

https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html


In [ ]:
%cd /content/drive/MyDrive/dlss24/

In [ ]:
%pwd

# CNN for image classification

In this HW you will do the following steps:


1.   *0.5 Pts*: Write pytorch architecture for ResNet (state of the art CNN architecture with high accuracy)
4.   *1.5 Pts*: Data pre process
5.   *1 Pts*: Fine Tune RestNet
7.   *2 Pts*: Classifying data
8.   *1 Pts*: Test algorithm


**Before you start, make sure you use the T4 GPU processor (Runtime->Change Runtime-> T4GPU)**


## *0.5 Pts*: Pytorch architecture for ResNet

We will use out-of-the-box ResNet from pytorch. To train the model we first definethe "train_model" function. Add the missing code:

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':

                  ### YOUR CODE STARTS HERE ###
                    ...  # Set model to training mode
                  ### YOUR CODE ENDS HERE ###

                else:

                  ### YOUR CODE STARTS HERE ###
                    ...  # Set model to evaluate mode
                  ### YOUR CODE ENDS HERE ###

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:

                  ### YOUR CODE STARTS HERE ###

                  #send the inputs and labels to the device (GPU)
                    inputs = ...
                    labels = ...

                  ### YOUR CODE ENDS HERE ###

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()

                            ### YOUR CODE STARTS HERE ###

                            #update the weights
                            ...

                            ### YOUR CODE ENDS HERE ###

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

## *1.5 Pts*: Data Preprocess


If you want, you can change the query made here, or add manually pictures to the folders. For the essay, this can be one of the ways to select your data.

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import random

politicians = {
    'SP': ['Christian Levrat', 'Simonetta Sommaruga', 'Alain Berset'],
    'SVP': ['Ueli Maurer', 'Christoph Blocher', 'Guy Parmelin'],
    'FDP': ['Ignazio Cassis', 'Johann Schneider-Ammann', 'Didier Burkhalter'],
    'CVP': ['Doris Leuthard', 'Viola Amherd', 'Gerhard Pfister'],
    'GLP': ['Martin Bäumle', 'Tiana Angelina Moser', 'Jürg Grossen']
}

# Define the ratio of data for training and validation
train_ratio = 0.8  # 80% for training
val_ratio = 0.2    # 20% for validation

# Create train and val directories
os.makedirs('data/train', exist_ok=True)
os.makedirs('data/val', exist_ok=True)

# Iterate over parties
for party, politicians_list in politicians.items():
    # Shuffle the list of politicians to randomize the data split
    random.shuffle(politicians_list)

    # Calculate the number of images for training and validation
    total_images = len(politicians_list)
    num_train_images = int(total_images * train_ratio)
    num_val_images = total_images - num_train_images

    # Create party directories in train and val directories
    os.makedirs(f'data/train/{party}', exist_ok=True)
    os.makedirs(f'data/val/{party}', exist_ok=True)

    # Iterate over politicians
    for i, politician in enumerate(politicians_list):
        # Determine whether to save the image in train or val directory
        if i < num_train_images:
            data_dir = 'train'
        else:
            data_dir = 'val'

        # Construct the Google search query URL
        query = f"{politician} {party} schweiz"
        url = f"https://www.google.com/search?q={query}&tbm=isch"

        # Send request to Google and parse the response
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        img_tags = soup.find_all("img")

        # Iterate over image tags and save the images
        for j, img_tag in enumerate(img_tags):
            image_url = img_tag["src"]
            if "http" not in image_url:
                continue
            try:
                # Download and save the image
                img_data = requests.get(image_url).content
                img = Image.open(BytesIO(img_data))
                img = img.convert("RGB")
                img.save(f"data/{data_dir}/{party}/{party}_{i}_{j}.jpg")
            except OSError:
                # Handle image processing errors
                print(f"Error processing image {image_url}")


In [ ]:
# 1 PTS: Create a function to eliminate duplicates in train folder:

In [ ]:
# 0.5 PTS: Data augmentation and normalization for training

data_transforms = {
    'train': transforms.Compose([
        ### YOUR CODE STARTS HERE ###

        # ... transformation for data augmentention 1
        # ... transformation for data augmentention 2
        # ... send to tensor
        # ... normalize

        ### YOUR CODE ENDS HERE ###
    ]),
    'val': transforms.Compose([
        ### YOUR CODE STARTS HERE ###

        # ... different size that training
        # ... repeat needed transformation to be consistent with training

        ### YOUR CODE ENDS HERE ###
    ]),
}

data_dir = 'data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes


Confirm your transformations:

In [ ]:
def imshow(inp, title=None):
    """Display image for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

## *1 Pts*: Fine Tune RestNet

Train the model from scratch (1) and fine tuned (2)

In [ ]:
# From scratch training (1)

### YOUR CODE STARTS HERE ###

model_scr = ... #initialize resnet18 model

# define how many classes your problem has
num_classes= ...


### YOUR CODE ENDS HERE ###

# number of features the ResNet architecture deals with
num_ftrs = model_scr.fc.in_features
model_scr.fc = nn.Linear(num_ftrs,num_classes)


In [ ]:

### YOUR CODE STARTS HERE ###
# specify the right loss function for classification
model_scr = ...

# specify the right loss function for classification
criterion = nn.CrossEntropyLoss()
### YOUR CODE ENDS HERE ###



In [ ]:

# Observe that all parameters are being optimized
optimizer_scr = optim.SGD(model_scr.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_scr, step_size=7, gamma=0.1)


In [ ]:
### YOUR CODE STARTS HERE ###
#use the first function defined: train_model
model_scr = ...

### YOUR CODE ENDS HERE ###

In [ ]:
# Fine Tuning(2)

### YOUR CODE STARTS HERE ###
#Re do the previous code but fine tuning the initial resnet model.
#Finetuning means starting with previously trained weights

....



### YOUR CODE ENDS HERE ###

## *2 Pts*: Classifying data

In [ ]:
# 1 PTS:  Print a confusion matrix from the best performance model you built

In [ ]:
# 1 PTS:  Define the steps for the best performer (data pre processing steps and model choices)
# interprete and justify the metrics

## *1 Pts*: Test algorithm

In [ ]:
# Build a function to predict a new image (choose/upload a new one that is not on train or validation folder)